In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, explode, split, mean, when, size,
    count, count_distinct, isnan, sum, avg,
    array_contains, broadcast, expr, round,
    min, max, desc, asc
)
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

from scripts import *

/Users/mykhailo/Desktop/big_data/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spark = SparkSession.builder \
    .appName("IMDb") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

25/01/20 12:28:28 WARN Utils: Your hostname, Mykhailos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.104 instead (on interface en0)
25/01/20 12:28:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/20 12:28:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
name_basics, title_akas, title_basics, title_crew, title_principals, title_ratings = load_tables(spark=spark, dirname='data')

## EDA

### name_basics

In [5]:
name_basics.show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|actor,miscellaneo...|tt0050419,tt00723...|
|nm0000002|      Lauren Bacall|     1924|     2014|actress,soundtrac...|tt0037382,tt00752...|
|nm0000003|    Brigitte Bardot|     1934|     NULL|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|       John Belushi|     1949|     1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00839...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,producer,...|tt0034583,tt00381...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,producer,mi...|tt0034583,tt00373...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,dir

In [4]:
analysis_basic_stats(df=name_basics, name="name_basics")

Table: name_basics

Row count: 14063748
Column count: 6
Column Types: DataFrame[nconst: string, primaryName: string, birthYear: int, deathYear: int, primaryProfession: string, knownForTitles: string]
Fully-null rows: 0 / 14063748 (0.00%)
Null-containing rows: 13870146 / 14063748 (98.62%)
Nulls per column:


25/01/19 20:42:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+-----------+---------+---------+-----------------+--------------+
|nconst|primaryName|birthYear|deathYear|primaryProfession|knownForTitles|
+------+-----------+---------+---------+-----------------+--------------+
|     0|         61| 13428602| 13825207|          2737102|       1592017|
+------+-----------+---------+---------+-----------------+--------------+

Summary Statistics:


+-------+---------+--------------------+------------------+------------------+--------------------+-------------------+
|summary|   nconst|         primaryName|         birthYear|         deathYear|   primaryProfession|     knownForTitles|
+-------+---------+--------------------+------------------+------------------+--------------------+-------------------+
|  count| 14063748|            14063698|            635146|            238541|            11326646|           12471731|
|   mean|     NULL|                 NaN|1953.7901175477764|1993.7733848688486|                NULL|               NULL|
| stddev|     NULL|                 NaN|35.724299912627465| 36.77411047300685|                NULL|               NULL|
|    min|nm0000001|!'aru Ikhuisi Pie...|                 4|                17|          accountant|          tt0000003|
|    max|nm9993719|  ﻿Thesia Kouloungou|              2024|              2025|writer,visual_eff...|tt9916856,tt4247168|
+-------+---------+--------------------+

### title_akas

In [6]:
title_akas.show()

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       1|          Carmencita|  NULL|    NULL|   original|                NULL|              1|
|tt0000001|       2|          Carmencita|    DE|    NULL|       NULL|       literal title|              0|
|tt0000001|       3|          Carmencita|    US|    NULL|imdbDisplay|                NULL|              0|
|tt0000001|       4|Carmencita - span...|    HU|    NULL|imdbDisplay|                NULL|              0|
|tt0000001|       5|          Καρμενσίτα|    GR|    NULL|imdbDisplay|                NULL|              0|
|tt0000001|       6|          Карменсита|    RU|    NULL|imdbDisplay|                NULL|              0|
|tt0000001|       7|          Карменс

In [7]:
analysis_basic_stats(df=title_akas, name="title_akas")

Table: title_akas



Row count: 50930086
Column count: 8
Column Types: DataFrame[titleId: string, ordering: int, title: string, region: string, language: string, types: string, attributes: string, isOriginalTitle: int]
Fully-null rows: 

0 / 50930086 (0.00%)
Null-containing rows: 

50929654 / 50930086 (100.00%)
Nulls per column:


+-------+--------+-----+--------+--------+--------+----------+---------------+
|titleId|ordering|title|  region|language|   types|attributes|isOriginalTitle|
+-------+--------+-----+--------+--------+--------+----------+---------------+
|      0|       0|   14|11427606|16798518|35327473|  50636498|            638|
+-------+--------+-----+--------+--------+--------+----------+---------------+

Summary Statistics:


+-------+---------+------------------+--------+--------+--------+-------------+------------------+------------------+
|summary|  titleId|          ordering|   title|  region|language|        types|        attributes|   isOriginalTitle|
+-------+---------+------------------+--------+--------+--------+-------------+------------------+------------------+
|  count| 50930086|          50930086|50930086|39502480|34131568|     15602613|            293588|          50929448|
|   mean|     NULL|4.2925066924096695|     NaN|    NULL|    NULL|         NULL|              NULL|0.2227624379514186|
| stddev|     NULL| 4.009950214127594|     NaN|    NULL|    NULL|         NULL|              NULL|0.4161001533152217|
|    min|tt0000001|                 1|       !|      AD|      af|  alternative|16mm release title|                 0|
|    max|tt9916880|               251|🧠+🧘=❤️|      ZW|      zu|workingvideo|     weekend title|                 1|
+-------+---------+------------------+--------+--------+--

### title_basics

In [8]:
title_basics.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   NULL|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|   NULL|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   NULL|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   NULL|             1|              

In [9]:
analysis_basic_stats(df=title_basics, name="title_basics")

Table: title_basics

Row count: 11345807
Column count: 9
Column Types: DataFrame[tconst: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: string, startYear: int, endYear: int, runtimeMinutes: int, genres: string]
Fully-null rows: 0 / 11345807 (0.00%)
Null-containing rows: 11285957 / 11345807 (99.47%)
Nulls per column:


+------+---------+------------+-------------+-------+---------+--------+--------------+------+
|tconst|titleType|primaryTitle|originalTitle|isAdult|startYear| endYear|runtimeMinutes|genres|
+------+---------+------------+-------------+-------+---------+--------+--------------+------+
|     0|        0|           8|            8|      1|  1418483|11211847|       7755109|501749|
+------+---------+------------+-------------+-------+---------+--------+--------------+------+

Summary Statistics:


+-------+---------+---------+--------------------+-------------+-------------------+------------------+------------------+-----------------+--------+
|summary|   tconst|titleType|        primaryTitle|originalTitle|            isAdult|         startYear|           endYear|   runtimeMinutes|  genres|
+-------+---------+---------+--------------------+-------------+-------------------+------------------+------------------+-----------------+--------+
|  count| 11345807| 11345807|            11345807|     11345807|           11345806|           9927324|            133960|          3590698|10844058|
|   mean|     NULL|     NULL|                 NaN|          NaN|0.14294744683630234|2006.0542246833083| 2007.304180352344|43.32685678383423|    NULL|
| stddev|     NULL|     NULL|                 NaN|          NaN| 14.830121196646218| 20.15505177286002|17.263732313290834|73.74252186193708|    NULL|
|    min|tt0000001|    movie|                   !|            !|                  0|              18

### title_crew

In [10]:
title_crew.show()

+---------+-------------------+---------+
|   tconst|          directors|  writers|
+---------+-------------------+---------+
|tt0000001|          nm0005690|     NULL|
|tt0000002|          nm0721526|     NULL|
|tt0000003|          nm0721526|     NULL|
|tt0000004|          nm0721526|     NULL|
|tt0000005|          nm0005690|     NULL|
|tt0000006|          nm0005690|     NULL|
|tt0000007|nm0005690,nm0374658|     NULL|
|tt0000008|          nm0005690|     NULL|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|     NULL|
|tt0000011|          nm0804434|     NULL|
|tt0000012|nm0525908,nm0525910|     NULL|
|tt0000013|          nm0525910|     NULL|
|tt0000014|          nm0525910|     NULL|
|tt0000015|          nm0721526|     NULL|
|tt0000016|          nm0525910|     NULL|
|tt0000017|nm1587194,nm0804434|     NULL|
|tt0000018|          nm0804434|     NULL|
|tt0000019|          nm0932055|     NULL|
|tt0000020|          nm0010291|     NULL|
+---------+-------------------+---

In [11]:
analysis_basic_stats(df=title_crew, name="title_crew")

Table: title_crew

Row count: 11345807
Column count: 3
Column Types: DataFrame[tconst: string, directors: string, writers: string]
Fully-null rows: 0 / 11345807 (0.00%)
Null-containing rows: 6404390 / 11345807 (56.45%)
Nulls per column:


+------+---------+-------+
|tconst|directors|writers|
+------+---------+-------+
|     0|  4910089|5568915|
+------+---------+-------+

Summary Statistics:


+-------+---------+---------+--------------------+
|summary|   tconst|directors|             writers|
+-------+---------+---------+--------------------+
|  count| 11345807|  6435718|             5776892|
|   mean|     NULL|     NULL|                NULL|
| stddev|     NULL|     NULL|                NULL|
|    min|tt0000001|nm0000005|           nm0000005|
|    max|tt9916880|nm9993709|nm9993713,nm54113...|
+-------+---------+---------+--------------------+



### title_principals

In [12]:
title_principals.show()

+---------+--------+---------+---------------+--------------------+--------------+
|   tconst|ordering|   nconst|       category|                 job|    characters|
+---------+--------+---------+---------------+--------------------+--------------+
|tt0000001|       1|nm1588970|           self|                NULL|      ["Self"]|
|tt0000001|       2|nm0005690|       director|                NULL|          NULL|
|tt0000001|       3|nm0005690|       producer|            producer|          NULL|
|tt0000001|       4|nm0374658|cinematographer|director of photo...|          NULL|
|tt0000002|       1|nm0721526|       director|                NULL|          NULL|
|tt0000002|       2|nm1335271|       composer|                NULL|          NULL|
|tt0000003|       1|nm0721526|       director|                NULL|          NULL|
|tt0000003|       2|nm1770680|       producer|            producer|          NULL|
|tt0000003|       3|nm0721526|       producer|            producer|          NULL|
|tt0

In [13]:
analysis_basic_stats(df=title_principals, name="title_principals")

Table: title_principals



Row count: 90076359
Column count: 6
Column Types: DataFrame[tconst: string, ordering: int, nconst: string, category: string, job: string, characters: string]
Fully-null rows: 

0 / 90076359 (0.00%)
Null-containing rows: 

90076359 / 90076359 (100.00%)
Nulls per column:


+------+--------+------+--------+--------+----------+
|tconst|ordering|nconst|category|     job|characters|
+------+--------+------+--------+--------+----------+
|     0|       0|     0|       0|73238562|  46423891|
+------+--------+------+--------+--------+----------+

Summary Statistics:


+-------+---------+-----------------+---------+--------+------------------+--------------------+
|summary|   tconst|         ordering|   nconst|category|               job|          characters|
+-------+---------+-----------------+---------+--------+------------------+--------------------+
|  count| 90076359|         90076359| 90076359|90076359|          16837797|            43652468|
|   mean|     NULL|7.009242358474991|     NULL|    NULL|1.3199999999999998|                NULL|
| stddev|     NULL| 5.15262614009433|     NULL|    NULL|0.7155417527999327|                NULL|
|    min|tt0000001|                1|nm0000001|   actor|"A Box in Town" by|             ["!CF"]|
|    max|tt9916880|               75|nm9993718|  writer|          écrivain|["🐑 Sheepish Bys...|
+-------+---------+-----------------+---------+--------+------------------+--------------------+



### title_ratings

In [14]:
title_ratings.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2113|
|tt0000002|          5.6|     285|
|tt0000003|          6.4|    2148|
|tt0000004|          5.3|     183|
|tt0000005|          6.2|    2871|
|tt0000006|          5.0|     205|
|tt0000007|          5.3|     897|
|tt0000008|          5.4|    2262|
|tt0000009|          5.4|     216|
|tt0000010|          6.8|    7813|
|tt0000011|          5.2|     408|
|tt0000012|          7.4|   13264|
|tt0000013|          5.7|    2037|
|tt0000014|          7.1|    6042|
|tt0000015|          6.1|    1245|
|tt0000016|          5.9|    1638|
|tt0000017|          4.6|     369|
|tt0000018|          5.2|     652|
|tt0000019|          5.2|      33|
|tt0000020|          4.7|     408|
+---------+-------------+--------+
only showing top 20 rows



In [15]:
analysis_basic_stats(df=title_ratings, name="title_ratings")

Table: title_ratings

Row count: 1518213
Column count: 3
Column Types: DataFrame[tconst: string, averageRating: double, numVotes: int]
Fully-null rows: 0 / 1518213 (0.00%)
Null-containing rows: 0 / 1518213 (0.00%)
Nulls per column:
+------+-------------+--------+
|tconst|averageRating|numVotes|
+------+-------------+--------+
|     0|            0|       0|
+------+-------------+--------+

Summary Statistics:
+-------+---------+------------------+------------------+
|summary|   tconst|     averageRating|          numVotes|
+-------+---------+------------------+------------------+
|  count|  1518213|           1518213|           1518213|
|   mean|     NULL| 6.951868808922076| 1026.915121922945|
| stddev|     NULL|1.3863973762204747|17784.266140391297|
|    min|tt0000001|               1.0|                 5|
|    max|tt9916880|              10.0|           2984904|
+-------+---------+------------------+------------------+



## General Analysis

### Title Types

In [39]:
# Title Type Distribution
title_type_counts = title_basics.groupBy("titleType").agg(count("tconst").alias("count")).orderBy(desc("count")).toPandas()

fig = px.bar(title_type_counts, x="titleType", y="count", title="Title Type Distribution", labels={"count": "Number of Titles"})
fig.show()

### Genres

In [37]:
# Genre Distribution
title_types = ["tvMovie", "movie", "tvShort", "short"]

genres_df = title_basics \
    .filter(col('titleType').isin(title_types)) \
    .filter(col("startYear").between(2000, 2024)) \
    .withColumn("genre", explode(split(col("genres"), ",")))

genre_counts = genres_df \
    .groupBy("genre") \
    .agg(count("tconst").alias("count")) \
    .orderBy(asc("count")) \
    .toPandas()

genre_counts = genre_counts[genre_counts["genre"] != "Short"]

fig = px.bar(
    genre_counts, 
    x="count", 
    y="genre", 
    orientation='h', 
    title="Genre Distribution in IMDb Dataset (tvMovie, movie, tvShort, short)",
    labels={"count": "Number of Titles Containing", "genre": "Genre"},
    color="count",
    color_continuous_scale='viridis'
)

fig.update_layout({
    'height': 800
})
fig.show()

### Yearly Movie Production

In [54]:
title_types = ["tvMovie", "movie", "tvShort", "short"]

yearly_counts = title_basics \
    .filter(col('titleType').isin(title_types)) \
    .withColumn("startYear", col("startYear").cast("int")) \
    .groupBy("startYear") \
    .agg(count("tconst").alias("count")) \
    .orderBy("startYear") \
    .toPandas()

fig = px.line(yearly_counts, x="startYear", y="count", title="Yearly Movie Production Trends", labels={"startYear": "Year", "count": "Number of Titles"})

fig.add_shape(
    go.layout.Shape(type="line", x0=2000, x1=2000, y0=0, y1=np.max(yearly_counts["count"]), line=dict(color="gray", width=2, dash="dash"))
)
fig.add_shape(
    go.layout.Shape(type="line", x0=2024, x1=2024, y0=0, y1=np.max(yearly_counts["count"]), line=dict(color="gray", width=2, dash="dash"))
)

fig.show()

### Rating Distribution

In [80]:
titles_filtered = title_basics \
    .filter(col('titleType').isin(title_types)) \
    .filter(col("startYear").between(2000, 2024)) \
    .select('tconst')

rating_counts = titles_filtered \
    .join(title_ratings, how='inner', on='tconst') \
    .select("averageRating") \
    .groupBy("averageRating") \
    .agg(count("averageRating").alias("count")) \
    .orderBy("averageRating") \
    .toPandas()


rating_counts["cumsum"] = rating_counts["count"].cumsum()

print(rating_counts["cumsum"].mean)

fig = px.bar(
    rating_counts,
    x="averageRating",
    y="count",
    title="IMDb Rating Distribution",
    labels={"averageRating": "Average Rating", "count": "Number of Titles"}
)

cumsum_mean = np.mean(rating_counts["cumsum"])
cumsum_mean_closest_index = (rating_counts["cumsum"] - cumsum_mean).abs().idxmin()
cumsum_mean_avg_rating = rating_counts['averageRating'][cumsum_mean_closest_index]

fig.add_trace(
    go.Scatter(x=rating_counts["averageRating"], y=rating_counts["cumsum"], mode='lines', name='Cumulative Sum', yaxis='y2')
)

fig.add_shape(
    go.layout.Shape(type="line", x0=cumsum_mean_avg_rating, x1=cumsum_mean_avg_rating, y0=0, y1=np.max(rating_counts['count'])*1.3, line=dict(color="lightgreen", width=2, dash="dash"))
)

fig.update_layout(
    yaxis2=dict(title="Cumulative Sum", overlaying='y', side='right')
)


fig.show()

<bound method Series.mean of 0        198
1        274
2        366
3        461
4        571
       ...  
86    344559
87    345331
88    346205
89    346529
90    347591
Name: cumsum, Length: 91, dtype: int64>


## Trends Analysis

In [16]:
trends_df = analysis_get_trends_dataframe(title_basics=title_basics, title_ratings=title_ratings)

### Popularity Trends

In [17]:
plot_genres_popularity(genre_trends_grouped=trends_df, top_n=10)

### Rating Trends

In [18]:
plot_genres_rating(genre_trends_grouped=trends_df, top_n=10)

### Interactive Plot (Popularity & Ratings)

In [20]:
plot_genres_interactive(genre_trends_pd=trends_df.toPandas())

25/01/19 21:12:12 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 156031 ms exceeds timeout 120000 ms
25/01/19 21:12:12 WARN SparkContext: Killing executors is not supported by current scheduler.
25/01/19 21:12:13 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$